In [13]:
import matplotlib.pyplot as plt
import numpy as np
from base64 import b64encode
import pandas as pd
from ipyleaflet import *
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
# !jupyter nbextension enable --py widgetsnbextension

In [16]:
#jupyter nbextension enable --py --sys-prefix ipyleaflet

In [17]:
import os
if not os.path.exists('wind-global.nc'):
    url = 'https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc'
    import requests
    r = requests.get(url)
    wind_data = r.content
    with open('wind-global.nc', 'wb') as f:
        f.write(wind_data)

import xarray as xr
ds = xr.open_dataset('wind-global.nc')

#pip install netcdf4

from ipyleaflet.velocity import Velocity
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(
    data=ds, 
    zonal_speed='u_wind', meridional_speed='v_wind', 
    latitude_dimension='lat', longitude_dimension='lon', 
    velocity_scale=0.01, max_velocity=20, 
    display_options=display_options
)

In [18]:
import requests
from datetime import datetime
front="https://www.n2yo.com/rest/v1/satellite/positions/"
end="/2/&apiKey=3J8GBR-WYHM3S-JHNLDD-4KER"
ID="25544/"  # satellite ID
# observer position information
obslatitude="41.702/"
obslongitude="-76.014/"
obsaltitude="0"
url=front+ID+obslatitude+obslongitude+obsaltitude+end
# url="https://www.n2yo.com/rest/v1/satellite/positions/25544/41.702/-76.014/0/2/&apiKey=3J8GBR-WYHM3S-JHNLDD-4KER"
r=requests.get(url)

raw_json=r.json()
timestamp = raw_json["positions"][0]["timestamp"]
ts = int(timestamp)
time=datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

satname=raw_json["info"]["satname"]
satlatitude=raw_json["positions"][0]["satlatitude"]
satlongitude=raw_json["positions"][0]["satlongitude"]
sataltitude=raw_json["positions"][0]["sataltitude"]

# print("time: "+time)
# print("satname: "+satname)
# print("latitude: "+str(satlatitude))
# print("longitude: "+str(satlongitude))
# print("altitude: "+str(sataltitude))

In [19]:
instance_id = '8090226b-e900-4993-bfa2-65f1d959b140'

sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + instance_id + '?showlogo=0&time=2020-10-02-12-00-00/2020-10-02-12-00-00'

# Orbital Eye

You can view the earth with an "eye" on satellites and see the wind direction and velocity on earth in the interative map below. You can use the dropdown menu to choose a satellite. If you see a black background, it is not an error! It means that the location of current satellite is at night. In addition, the moving objects are not meteors but represent the movement of winds.

In [24]:
m = Map(center=(20, 10), zoom=10)
#m.add_layer(WMSLayer(url=sh_wms_url, layers="TRUE_COLORSENTINEL-2-L1C", tile_size=512))
m.add_layer(wind)

#@interact
#def location(latitude='50', longitude='15'):
    
def info_select(satellite):

    front="https://www.n2yo.com/rest/v1/satellite/positions/"
    end="/4/&apiKey=3J8GBR-WYHM3S-JHNLDD-4KER"
    if satellite == 'SENTINEL_1':  
        ID="39634/"  # satellite ID
    elif satellite == 'SENTINEL_2A':  
        ID="40697/"  # satellite ID
    elif satellite == 'SENTINEL_2B':  
        ID="42063/"  # satellite ID
    elif satellite == 'SENTINEL_3':  
        ID="41335/"  # satellite ID
    elif satellite == 'SENTINEL_5P':  
        ID="42969/"  # satellite ID
    elif satellite == 'LANDSAT_8':  
        ID="39084/"  # satellite ID
    url=front+ID+obslatitude+obslongitude+obsaltitude+end
    # url="https://www.n2yo.com/rest/v1/satellite/positions/25544/41.702/-76.014/0/2/&apiKey=3J8GBR-WYHM3S-JHNLDD-4KER"
    r=requests.get(url)

    raw_json=r.json()
    timestamp = raw_json["positions"][0]["timestamp"]
    ts = int(timestamp)
    time=datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

    satname=raw_json["info"]["satname"]
    satlatitude=raw_json["positions"][0]["satlatitude"]
    satlongitude=raw_json["positions"][0]["satlongitude"]
    #sataltitude=raw_json["positions"][0]["sataltitude"]
    
    layer = satellite
    #date = pd.Timestamp(date)
    sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + instance_id + '?showlogo=0&time={}/{}'.format(str(time)[:-9],str(time)[:-9])
    m.add_layer(WMSLayer(url=sh_wms_url, layers='{}'.format(layer), tile_size=512))
    #m.add_layer(Marker(location=(int(latitude), int(longitude))))
    m.center=(satlatitude,satlongitude)
    print("time: "+time)
    print("latitude: "+str(satlatitude))
    print("longitude: "+str(satlongitude))

    return m

_ = interact(info_select, satellite = widgets.Dropdown(options=['SENTINEL_1','SENTINEL_2A', 'SENTINEL_2B','SENTINEL_3','SENTINEL_5P','LANDSAT_8']))
            # layer = widgets.Dropdown(options=['FALSE_COLOR', 'TRUE_COLOR']),
            # date = widgets.DatePicker(value=pd.to_datetime('2020-10-01'))


interactive(children=(Dropdown(description='satellite', options=('SENTINEL_1', 'SENTINEL_2A', 'SENTINEL_2B', '…